In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from csv import reader
from datetime import datetime

In [ ]:
activities = {'Sleeping': 0,  'Browsing':   1, 'Socializing': 2,  'Education': 3,
              'Homework': 4,  'Eating':     5, 'Shower':      6,  'Work':      7,
              'Travel':   8,  'Exercising': 9, 'Job Search':  10, 'Guitar':    11, 
              'Darts':    12, 'Robotics':   13}
week = {'MONDAY': 1, 'TUESDAY': 2, 'WEDNESDAY': 3, 'THURSDAY': 4, 'FRIDAY': 5, 'SATURDAY': 6, 'SUNDAY': 7}
months = {'JANUARY': 1, 'FEBRUARY': 2, 'MARCH': 3, 'APRIL': 4, 'MAY': 5, 'JUNE': 6, 
          'JULY': 7, 'AUGUST': 8, 'SEPTEMBER': 9, 'OCTOBER': 10, 'NOVEMBER': 11, 'DECEMBER': 12}

In [ ]:
with open('hours.csv', 'r') as f:
    csv_reader = reader(f)
    df = []
    year = month = day = weekday = None
    actions = np.empty(288)
    actions.fill(-1)
    previous_day = True
    for row in csv_reader:
        try:
            if row[0] == '' or row[0] == 'Total':
                # print(actions)
                continue
            if row[-1] == '':
                df += [[year, month, day, weekday] + list(actions)]
                year = row[2]
                month = months[row[0].split(' ')[1]]
                day = int(row[0].split(' ')[2])
                weekday = week[row[0].split(',')[0]]
            else:
                activity = activities[row[0]]
                start_time = row[1].split(':')
                start_time = int(start_time[0])*12 + int(start_time[1])//5
                end_time = row[2].split(':')
                end_time = int(end_time[0])*12 + int(end_time[1])//5

                if end_time == 0:
                    end_time = 288

                if end_time < start_time:
                    if previous_day:
                        end_time = 288
                    else:
                        start_time = 0
                    previous_day = not previous_day

                actions[start_time:end_time] = activity
        except:
            print(row)

df = pd.DataFrame(df[1:], columns = ['year', 'month', 'day', 'day_of_week'] + [str(i//12).zfill(2)+':'+str(i%12 *5).zfill(2) for i in range(288)])

In [ ]:
print(df.shape)
# print(df.head())
# print(df['00:00'].value_counts())

In [ ]:
# Plot the stacked area chart for a specific timeframe
freq_df = pd.DataFrame(index = activities.values())
for col in df.columns[4:]:
    freqs = df[col].value_counts()
    freq_df = pd.concat([freq_df, freqs], axis=1)
freq_df = freq_df.fillna(0)

In [ ]:
# Verify total activity counts
for row in range(freq_df.shape[0]):
    print(row, list(activities.keys())[row], np.sum(freq_df.iloc[row]))

In [ ]:
plt.rcParams["figure.figsize"] = (20, 13)
plt.stackplot(df.columns[4:], freq_df.iloc[0], freq_df.iloc[1],
              freq_df.iloc[2], freq_df.iloc[3], freq_df.iloc[4],
              freq_df.iloc[5], freq_df.iloc[6], freq_df.iloc[7],
              freq_df.iloc[8], freq_df.iloc[9], freq_df.iloc[10],
              freq_df.iloc[11], freq_df.iloc[12], freq_df.iloc[13], 
              labels=activities.keys())
plt.legend()
plt.xticks(df.columns[4::12])
plt.xlim([0, 287])
plt.show()

In [ ]:
df.to_csv('output.csv')